In [1]:
import pandas as pd
#outsiders se genera con: python3 run_all.py --option 0 --contract_type outsiders --save_path descargas_9_jun
outsiders_path='/home/sblanco/NextProc/PLACE/outsiders.parquet'

#insiders se genera con:  python3 run_all.py --option 0 --contract_type insiders --save_path /home/sblanco/NextProc/PLACE/
insiders_path = '/home/sblanco/NextProc/PLACE/insiders.parquet'

year = 2024


In [2]:
print ('Leyendo datos...')
df_PLACE_out = pd.read_parquet(outsiders_path)
df_PLACE_insiders = pd.read_parquet(insiders_path)

Leyendo datos...


In [3]:
print ('Unificando datos...')


def unify_colname(col):
    return ".".join([el for el in col if el])

df_PLACE_out = df_PLACE_out.rename(columns={"id":"place_id"})
df_PLACE_insiders = df_PLACE_insiders.rename(columns={"id":"place_id"})

df_PLACE_out.columns = [unify_colname(col) for col in df_PLACE_out.columns]
df_PLACE_insiders.columns = [unify_colname(col) for col in df_PLACE_insiders.columns]


Unificando datos...


In [4]:
# outsiders
columns_keep_out = [
    #'ContractFolderStatus.ContractFolderID', # Expediente
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', #CPV
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode', # Country NUTS
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name', # Ubicación orgánica
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount', # Presupuesto
    'ContractFolderStatus.ContractFolderID', #Expediente
    'title', # PLACE title (objective)
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate', # Date
    'place_id',
    'link',
    'ContractFolderStatus.TechnicalDocumentReference.Attachment.ExternalReference.URI', #pliego técnico  
    'ContractFolderStatus.LegalDocumentReference.Attachment.ExternalReference.URI', #pliego administrativo
    'ContractFolderStatus.UUID' # referencia a TED
]

#insiders:
columns_keep_ins = [
    #'ContractFolderStatus.ContractFolderID', # Expediente
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', # CPV
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode', # Country NUTS
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name', # Ubicación orgánica
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount', # Presupuesto
    'ContractFolderStatus.ContractFolderID', #Expediente
    'title', # PLACE title (objective)
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate', # Date
    'place_id',
    'link',
    'ContractFolderStatus.TechnicalDocumentReference.Attachment.ExternalReference.URI', #pliego técnico  
    'ContractFolderStatus.LegalDocumentReference.Attachment.ExternalReference.URI', #pliego administrativo
    'ContractFolderStatus.UUID' # referencia a TED    
]


df_PLACE_out = df_PLACE_out[columns_keep_out]
df_PLACE_insiders = df_PLACE_insiders[columns_keep_ins]

df_PLACE_all ={ 'outsiders': {'name':'outsiders','data':df_PLACE_out,'total':df_PLACE_out.shape[0] },
                'insiders': {'name':'insiders','data':df_PLACE_insiders,'total':df_PLACE_insiders.shape[0] }                  
              }

for k,v in df_PLACE_all.items():
    v['data']
    v['data'] = v['data'].rename(columns={
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode':'cpv',
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode':'nuts',
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name':'org_loc',
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount':'budget',
    'ContractFolderStatus.ContractFolderID':'expedient',
    'title':'objective',
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate':'place_date',
    'place_id':'place_id',
    'link':'link',
    'ContractFolderStatus.UUID':'tedUUID',
    'ContractFolderStatus.TechnicalDocumentReference.Attachment.ExternalReference.URI':'pliego_tecnico',
    'ContractFolderStatus.LegalDocumentReference.Attachment.ExternalReference.URI':'pliego_administrativo'    
})


In [5]:
print ('Unificando fechas')
for k,v in df_PLACE_all.items():
    print ('Para %s' % (v['name']))
    v['data']['date'] = v['data']['place_date'].astype(str)
    v['data']['date'] = v['data']['date'].str.strip('[]').str.strip('"[').str.split(' ')
    v['data']['date'] = v['data']['date'].apply(lambda x: str(x[0].strip().strip("'").strip("',").strip('"]') if isinstance(x, list) else x.strip().strip("',").strip('"]')))

Unificando fechas
Para outsiders
Para insiders


In [6]:

for k,v in df_PLACE_all.items():
    #quitamos ids duplicados:
    v['data'].drop_duplicates(subset=['place_id'],inplace=True)
    unique = v['data'].shape[0]
    #filtramos por el año seleccionado:
    v['data'] = v['data'][v['data']['date'].str.contains(str(year))]    
    print ('**  %s: ** \n \t \t Hay %s licitaciones, de los cuales, únicos son %s. \n \t \t \t \t ->Si filtramos para %s nos quedamos sólo con %s licitaciones únicas \n'
           % (v['name'], f"{v['total']:,}", f"{unique:,}", year, f"{v['data'].shape[0]:,}")
          )

**  outsiders: ** 
 	 	 Hay 368,831 licitaciones, de los cuales, únicos son 368,831. 
 	 	 	 	 ->Si filtramos para 2024 nos quedamos sólo con 51,877 licitaciones únicas 

**  insiders: ** 
 	 	 Hay 868,865 licitaciones, de los cuales, únicos son 868,865. 
 	 	 	 	 ->Si filtramos para 2024 nos quedamos sólo con 156,105 licitaciones únicas 



In [ ]:
for k,v in df_PLACE_all.items():
    v['listOfPliego_tecnico'] = v['data'][v['data']['pliego_tecnico']!='<NA>']['pliego_tecnico'].to_list()
    v['numOfPliego_tecnico'] = len (v['listOfPliego_tecnico'])
    v['numOfUniquePliego_tecnico'] = len (set (v['listOfPliego_tecnico']))

    v['listOfPliego_administrativo'] = v['data'][v['data']['pliego_administrativo']!='<NA>']['pliego_administrativo'].to_list()
    v['numOfPliego_administrativo'] = len (v['listOfPliego_administrativo'])
    v['numOfUniquePliego_administrativo'] = len (set (v['listOfPliego_administrativo']))

    v['listOfTeds'] = v['data'][v['data']['tedUUID']!='<NA>']['tedUUID'].to_list()
    v['numOfTeds'] = len (v['listOfTeds'])
    v['numOfUniqueTed'] = len (set (v['listOfTeds']))
    
    print ('** %s: ** \n \t \t Para las licitaciones únicas del año %s, hay %s identificadores de TED, de los cuáles son únicos %s' %
            (v['name'], year,    f"{v['numOfTeds']:,}" ,  f"{v['numOfUniqueTed']:,}")
          )
    print ('\t  \t Para las licitaciones únicas del año %s, hay %s pliegos técnicos, de los cuáles son únicos %s' %
            ( year,    f"{v['numOfPliego_tecnico']:,}"  ,  f"{v['numOfUniquePliego_tecnico']:,}")
          )
    print ('\t  \t Para las licitaciones únicas del año %s, hay %s pliegos administrativos, de los cuáles son únicos %s' %
            ( year,   f"{v['numOfPliego_administrativo']:,}" , f"{v['numOfUniquePliego_administrativo']:,}" )
          )



In [ ]:
import matplotlib.pyplot as plt

for k,v in df_PLACE_all.items():
    df = v['data']
    print ('Relación UUID Ted y presupuesto de la licitación, para el año %s y para %s.' % (year, v['name']))
    df['budget_bin'] = pd.qcut(df['budget'], q=20)
    df['budget_bin'] = df['budget_bin'].apply(lambda x: f"{int(x.left):,} - {int(x.right):,}")
    total_counts = df.groupby('budget_bin').size()
    ted_counts = df[df['tedUUID'] != '<NA>'].groupby('budget_bin').size()
    counts = pd.DataFrame({'Total': total_counts, 'ted ≠ NA': ted_counts}).fillna(0)
    counts.plot(kind='bar', figsize=(10,6))
    plt.ylabel('Count')
    plt.xlabel('Budget Range')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
#COMPROBAR CUÁNTOS TIENEN TED Y PLIEGOS ADMINISTRATIVOS Y TECNICOS A LA VEZ!!!!!!
for k,v in df_PLACE_all.items():
    out_df = v['data']
    data_red = out_df[out_df['tedUUID']!='<NA>']
    numteds = data_red.shape[0]
    data_red = data_red [ data_red [ 'pliego_tecnico' ] !='<NA>']
    ted_and_pliegotec = data_red.shape[0]
    data_red = data_red [ data_red [ 'pliego_administrativo' ] !='<NA>']
    ted_and_pliegotec_and_pliegoadmin = data_red.shape[0]
    print ('** %s: ** \n ' %v['name'])
    print ('\t  \t Para las licitaciones únicas del año %s, hay %s identficadores ted, de los cuáles:' \
            '\n \t \t \t %s cuentan con pliegos técnicos y %s cuentan con pliegos ténicos y administrativos' %
            ( year,   f"{numteds:,}" , f"{ted_and_pliegotec:,}", f"{ted_and_pliegotec_and_pliegoadmin:,}" )
          )
    

In [ ]:
import matplotlib.pyplot as plt

for k,v in df_PLACE_all.items():
    df = v['data']
    print ('Relación Pliegos Tec y presupuesto de la licitación, para el año %s y para %s.' % (year, v['name']))
    df['budget_bin'] = pd.qcut(df['budget'], q=20)
    df['budget_bin'] = df['budget_bin'].apply(lambda x: f"{int(x.left):,} - {int(x.right):,}")
    total_counts = df.groupby('budget_bin').size()
    pliego_counts = df[df['pliego_tecnico'] != '<NA>'].groupby('budget_bin').size()
    counts = pd.DataFrame({'Total': total_counts, 'pliegostec ≠ NA': pliego_counts}).fillna(0)
    counts.plot(kind='bar', figsize=(10,6))
    plt.ylabel('Count')
    plt.xlabel('Budget Range')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


#pliego_tecnico

In [ ]:
#################### FIN DEL NOTEBOOK, POR DEBAJO DE AQUÍ SÓLO HAY HORRORES.

In [ ]:
df_PLACE_all['outsiders']['data']

In [7]:
df = df_PLACE_all['outsiders']['data'].copy()
df = df[df['pliego_tecnico'] != '<NA>']
df.shape[0]

35340

In [8]:
def createUrls ( row ):
    row['url'] = {'tecnico':row['pliego_tecnico'],'administrativo':row['pliego_administrativo']}
    return row

In [9]:
df =df.apply(createUrls, axis=1).reset_index()

In [12]:
import numpy as np
result = df[['place_id', 'url']]
result['id'] = np.arange(0, len(result))
result.to_parquet ('/export/usuarios01/sblanco/red_data_outsiders_2024.parquet')
result

/tmp/ipykernel_2754759/2315884782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['id'] = np.arange(0, len(result))


,place_id,url,id
0,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,0
1,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://contractaciopublica.cat/p...,1
2,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,2
3,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,3
4,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://contratos-publicos.comuni...,4
...,...,...,...
35335,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://contractaciopublica.cat/p...,35335
35336,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,35336
35337,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,35337
35338,https://contrataciondelestado.es/sindicacion/P...,{'tecnico': 'https://www.contratacion.euskadi....,35338


In [ ]:
data = df_PLACE_all['outsiders']['data'].copy()
data['budget'].value_counts().to_csv('mollete.csv')

In [ ]:
'''Licitaciones por tipo de contrato:
Tipo de Contrato	División Principal del CPV	Descripción General
Obras	45	Trabajos de construcción y obras de ingeniería civil.
Suministros	03-44 (aprox.)	Compra de todo tipo de productos y bienes.
Servicios	50-98	Prestación de actividades (limpieza, consultoría, transporte, etc.).
'''
for k,v in df_PLACE_all.items():
    out_df = v['data']
    num_obras = out_df.query ('cpv.str.startswith("45")').shape[0]
    num_suministro = out_df.query ('cpvout_df.query ('cpv.str.startswith("45")').shape[0]
    num_servicio = 
#display(dataFrame.query('Salary  <= 100000 & Age < 40 & JOB.str.startswith("C").values'))

data_red = out_df.query ('budget < 40000 & tedUUID !="<NA>"')
data_red['budget']



In [ ]:
def getTipoContrato (row):
    data = str(row.tolist()[0][0])[:2].replace('[','')
    tipo = '<NA>'
    if data == 'na':        
        tipo = '<NA>'
    else:
        if int(data) in range (49,99):
            tipo = 'servicios'
        else:
            if int(data) in range (2,51):
                tipo = 'suministros'
            else:
                if int(data)==45:
                    tipo = 'obras'
    return tipo


data = df_PLACE_all['outsiders']['data'].copy()
data['tipocontrato'] = data.apply (getTipoContrato, axis=1)

In [ ]:
#data[data'tipocontrato']!='<NA>'].iloc[57]

row = data [ data [ 'tipocontrato' ] =='<NA>'].iloc[0]
int (str(row.tolist()[0][0])[:2].replace('[','')) 
#int (str(row.tolist()[0][0])[:2].replace('[','')) 
row.cpv.tolist()[0].split(',')[0]

In [ ]:
row


In [ ]:
data = df_PLACE_all['outsiders']['data']['cpv'].iloc[4000]
data = data.tolist()[0][:2]
if int(data) in range (50,98):
    tipo = 'servicios'
else:
    if int(data) in range (3,44):
        tipo = 'suministros'
    else:
        if int(data)==45:
            tipo = 'obras'


print ('%s -> %s' % (tipo, data))
#.query ('cpv.str.startswith("45")')

In [ ]:
data = ['[39298900, 31527200]']
data[0].split(',')

In [ ]:
out_df

In [ ]:
df = df_PLACE_all['outsiders']['data']

In [ ]:
#df  .query ('budget < 40000 & tedUUID !="<NA>"')
import matplotlib.pyplot as plt
df['budget_bin'] = pd.qcut(df['budget'], q=10)
df[df['tedUUID'] !="<NA>"].groupby ('budget_bin').size().plot(kind="bar")

plt.show()

In [ ]:
df['budget_bin'] = pd.qcut(df['budget'], q=20)
df['budget_bin'] = df['budget_bin'].apply(lambda x: f"{int(x.left):,} - {int(x.right):,}")
total_counts = df.groupby('budget_bin').size()
ted_counts = df[df['tedUUID'] != '<NA>'].groupby('budget_bin').size()
counts = pd.DataFrame({'Total': total_counts, 'ted ≠ NA': ted_counts}).fillna(0)
counts.plot(kind='bar', figsize=(10,6))
plt.ylabel('Count')
plt.xlabel('Budget Range')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df2 = df_PLACE_all['insiders']['data']

df2['budget_bin'] = pd.qcut(df2['budget'], q=20)
df2['budget_bin'] = df2['budget_bin'].apply(lambda x: f"{int(x.left):,} - {int(x.right):,}")
total_counts = df2.groupby('budget_bin').size()
ted_counts = df2[df2['tedUUID'] != '<NA>'].groupby('budget_bin').size()
counts = pd.DataFrame({'Total': total_counts, 'ted ≠ NA': ted_counts}).fillna(0)
counts.plot(kind='bar', figsize=(10,6))
plt.ylabel('Count')
plt.xlabel('Budget Range')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df['budget'].value_counts()

In [ ]:
out_df = df_PLACE_all['outsiders']['data']

#display(dataFrame.query('Salary  <= 100000 & Age < 40 & JOB.str.startswith("C").values'))

data_red = out_df.query ('budget < 40000 & tedUUID !="<NA>"')
data_red['budget']

In [ ]:
dat

In [ ]:

df_PLACE_all['insiders']['listOfPliego_administrativo']

In [ ]:
df_PLACE_minors.columns

In [ ]:
prueba = df_PLACE_all['outsiders']['data'].copy()

In [ ]:
prueba['tedUUID'].value_counts()
filter_df = prueba[prueba['tedUUID']!='<NA>']['tedUUID'].to_list()

In [ ]:
len(filter_df)

In [ ]:
len(set (filter_df))

In [ ]:
prueba.shape[0]

In [ ]:
prueba['place_date']


In [ ]:
df_PLACE_all['outsiders']['data']['place_id']

In [ ]:
#limpiar por id y quedarnos con unicos y para el 2024

In [ ]:
#pliego tecnico
df_PLACE_out[['ContractFolderStatus.TechnicalDocumentReference.Attachment.ExternalReference.URI']].value_counts()

In [ ]:
df_PLACE_out[['ContractFolderStatus.UUID']].value_counts()

In [ ]:
#pliego administrativo 
df_PLACE_out[['ContractFolderStatus.LegalDocumentReference.Attachment.ExternalReference.URI']].value_counts()

In [ ]:
# outsiders
columns_keep_out = [
    #'ContractFolderStatus.ContractFolderID', # Expediente
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', #CPV
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode', # Country NUTS
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name', # Ubicación orgánica
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount', # Presupuesto
    'ContractFolderStatus.ContractFolderID', #Expediente
    'title', # PLACE title (objective)
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate', # Date
    'origin',
    'place_id',
    'link'
]

In [ ]:
df_PLACE_out